In [127]:
from google_drive_downloader import GoogleDriveDownloader as gdd

gdd.download_file_from_google_drive(file_id='1ZKrbsUlrsTgJSlFMz2EGmTINFCQ8PcjO',
dest_path='content/concrete_crack_data.zip',
unzip=True)

In [128]:
import pandas as pd # Data analysis and manipultion tool
import numpy as np # Fundamental package for linear algebra and multidimensional arrays
import tensorflow as tf # Deep Learning Tool
import os # OS module in Python provides a way of using operating system dependent functionality
import cv2 # Library for image processing
from sklearn.model_selection import train_test_split # For splitting the data into train and validation set
from sklearn.metrics import f1_score

In [129]:
data=[]
img_size=100 # Here we are taking image size as 100, but it's on you. You can take 50 or 40 or 32 and so on
def create_data():
    for item in ['Negative','Positive']:
        path='/content/content/concrete_cracked_images/train/'+item+"/"

        for img in os.listdir(path): # os.listdir gets you all the list of name of files located in the given path
            try:
                img_array=cv2.imread(os.path.join(path,img),cv2.IMREAD_GRAYSCALE) # converts the image to pixels and gray scales the images
                new_img_array=cv2.resize(img_array,(img_size,img_size)) # resizing the images
                if item == 'Negative':
                    data.append([new_img_array,0])
                else:
                   data.append([new_img_array, 1]) # appending the list of image pixels and respective target value (i.e. animal type) in data
            except Exception as e:
                    pass # try and except is exception handling case in python, saves you from getting errors


create_data()

In [130]:
len(data)

28718

In [131]:
# image pixels of a image
data[0]

[array([[164, 167, 169, ..., 158, 153, 156],
        [164, 165, 170, ..., 156, 151, 153],
        [158, 158, 166, ..., 157, 151, 147],
        ...,
        [162, 163, 158, ..., 153, 157, 158],
        [164, 163, 158, ..., 155, 161, 158],
        [161, 164, 166, ..., 155, 156, 152]], dtype=uint8), 0]

In [132]:
np.random.shuffle(data)

In [133]:
x = []
y = []
for image in data:
    x.append(image[0])
    y.append(image[1])

# converting x & y to numpy array as they are list
x = np.array(x)
y = np.array(y)

In [134]:
np.unique(y, return_counts=True)

(array([0, 1]), array([13818, 14900]))

In [135]:
# split the data
X_train, X_val, y_train, y_val = train_test_split(x,y,test_size=0.3, random_state = 42)

In [136]:
# Defining the model
model = tf.keras.Sequential([
tf.keras.layers.Flatten(input_shape=(100, 100)), # flattening the image
tf.keras.layers.Dense(100, activation='relu'),
tf.keras.layers.Dense(100, activation='relu'),
tf.keras.layers.Dense(100, activation='relu'),
tf.keras.layers.Dense(50, activation='relu'),
tf.keras.layers.Dense(1, activation='sigmoid')
])

model.compile(optimizer='adam',
loss='binary_crossentropy',
metrics=['accuracy'])

model.fit(X_train, y_train, epochs=10, batch_size=50)

Epoch 1/10
403/403 [==============================] - 6s 13ms/step - loss: 57.1607 - accuracy: 0.5477
Epoch 2/10
403/403 [==============================] - 5s 13ms/step - loss: 5.4181 - accuracy: 0.6319
Epoch 3/10
403/403 [==============================] - 5s 13ms/step - loss: 0.3856 - accuracy: 0.8385
Epoch 4/10
403/403 [==============================] - 5s 13ms/step - loss: 1.2770 - accuracy: 0.7666
Epoch 5/10
403/403 [==============================] - 5s 13ms/step - loss: 0.6922 - accuracy: 0.5176
Epoch 6/10
403/403 [==============================] - 5s 13ms/step - loss: 0.6920 - accuracy: 0.5171
Epoch 7/10
403/403 [==============================] - 5s 13ms/step - loss: 0.6922 - accuracy: 0.5213
Epoch 8/10
403/403 [==============================] - 5s 13ms/step - loss: 0.6926 - accuracy: 0.5174
Epoch 9/10
403/403 [==============================] - 5s 13ms/step - loss: 0.6925 - accuracy: 0.5186
Epoch 10/10
403/403 [==============================] - 5s 13ms/step - loss: 0.6920 - accur

In [137]:
model.evaluate(X_val, y_val)

270/270 [==============================] - 1s 4ms/step - loss: 0.6924 - accuracy: 0.5183


[0.692409873008728, 0.5183379650115967]

In [138]:
test_image_order = pd.read_csv("/content/content/concrete_cracked_images/Testing_set_concrete_crack.csv")
test_image_order.head()

,filename
0,Image_1.jpg
1,Image_2.jpg
2,Image_3.jpg
3,Image_4.jpg
4,Image_5.jpg


In [139]:
file_paths = [[fname, '/content/content/concrete_cracked_images/test/' + fname] for fname in test_image_order['filename']]

In [140]:
# Confirm if number of images is same as number of labels given
if len(test_image_order) == len(file_paths):
  print('Number of image names i.e. ', len(test_image_order), 'matches the number of file paths i.e. ', len(file_paths))
else:
  print('Number of image names does not match the number of filepaths')

Number of image names i.e.  11282 matches the number of file paths i.e.  11282


In [141]:
test_images = pd.DataFrame(file_paths, columns=['filename', 'filepaths'])
test_images.head()

,filename,filepaths
0,Image_1.jpg,/content/content/concrete_cracked_images/test/...
1,Image_2.jpg,/content/content/concrete_cracked_images/test/...
2,Image_3.jpg,/content/content/concrete_cracked_images/test/...
3,Image_4.jpg,/content/content/concrete_cracked_images/test/...
4,Image_5.jpg,/content/content/concrete_cracked_images/test/...


In [142]:
test_pixel_data = [] # initialize an empty numpy array
image_size = 100 # image size taken is 100 here. one can take other size too
for i in range(len(test_images)):
    img_array = cv2.imread(test_images['filepaths'][i], cv2.IMREAD_GRAYSCALE) # converting the image to gray scale
    new_img_array = cv2.resize(img_array, (image_size, image_size)) # resizing the image array
    test_pixel_data.append(new_img_array)
test_pixel_data = np.array(test_pixel_data)

In [143]:
test_pixel_data = np.array(test_pixel_data)

In [144]:
pred = model.predict(test_pixel_data)

In [145]:
# The predicted values are the probabilities value
pred[0]

array([0.51583976], dtype=float32)

In [146]:
prediction = []
for value in pred:
    if value < 0.5:
        prediction.append("Negative")
    else:
        prediction.append("Positive")

In [147]:
res = pd.DataFrame({'prediction':prediction})

In [148]:
from google.colab import files

In [149]:
res.to_csv("ConcretePrediction.csv", index = False)

In [150]:
#files.download('ConcretePrediction.csv')

In [151]:
from google.colab import files
files.download('ConcretePrediction.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>